<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/Society_Repository.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Authentication**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re

from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def cleanText(row):
  return row.str.upper().str.strip()

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          #print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.5 MB/s eta 0:00:00


In [ ]:
#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

def get_data_df1(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[2:], columns = l[1])
  return ldf

In [ ]:
daysDelta =0

DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')

Last30DaysDate = (datetime.today() - timedelta(days = 30 + daysDelta)).date()
Last60DaysDate = (datetime.today() - timedelta(days = 60 + daysDelta)).date()
Last90DaysDate = (datetime.today() - timedelta(days = 90 + daysDelta)).date()
Last7DaysDate = (datetime.today() - timedelta(days = 7 + daysDelta)).date()
Last45DaysDate = (datetime.today() - timedelta(days = 45 + daysDelta)).date()
MonthStartDate = datetime.today().date().replace(day=1)

start_date = Last30DaysDate.strftime('%d %B %Y')

file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Apr 2023/"

#**Data Import**

In [ ]:
key = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
range = "Builder wise"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
builder_soc = get_data_df(key, range)

In [ ]:
builder_soc = builder_soc[['project name','builder','Locality','Latitude','Longitude','possession date']]

In [ ]:
builder_soc.rename({'project name':'society','Locality':'locality','Latitude':'latitude','Longitude':'longitude'}, inplace=True, axis =1)

In [ ]:
builder_soc.drop_duplicates(subset = 'society',inplace=True)

In [ ]:
# builder_soc['Cleaned Society Name'] = builder_soc['society'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')

In [ ]:
builder_soc

,society,builder,locality,latitude,longitude,possession date
0,5th Evergreen Apartments,5th Angle Constructions,Porur,13.03835448,80.15637366,|Possession started from: Jan'22
1,A K Navratna,A K Builders,East Tambaram,12.92383532,80.13062962,|Possession started from: Aug'22
2,Blooming Garden,A S Construction Chennai,Thalambur,12.8544382,80.2066887,|Possession started from: Feb'22
3,A Square Singapor,A Square Housing Development Private Limited,Kadambathur,13.11277332,79.86653181,Possession: Mar-2021
4,Singapor Township,A Square Housing Development Private Limited,Tiruvallur,13.12338779,79.91212348,Possession: Dec-2020
...,...,...,...,...,...,...
4479,Viva Vittal,,Pallavaram,12.96767184,80.14926638,
4480,VSS Vishwaraja Apartments,,Palavakkam,12.96161554,80.25622412,Possession: Aug-2019
4481,VSU Dhayal Villa,,Thoraipakkam,12.94143406,80.2358655,Possession: Dec-2016
4482,Woddies Alwarthirunagar Flats,,Porur,13.03835448,80.15637366,


In [ ]:
key = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
range = "NB.com listing"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
nb = get_data_df(key, range)

In [ ]:
from ast import literal_eval
nb['amenities'] = nb['amenities'].replace("",'{}')


In [ ]:
import yaml
nb['amenities'] = nb['amenities'].apply(lambda x: yaml.load(x, Loader=yaml.Loader))

In [ ]:
nb_sub = pd.json_normalize(nb['amenities'])

In [ ]:
nb_sub

,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,POOL,CPA,FS,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,True,True,False,True,True,True,True,False,True,False,True,False,True,False,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,False,True,False,False,True,True,False,True,True,False
3,False,False,False,False,False,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False
4,True,False,True,False,False,True,False,False,True,False,True,False,False,False,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False
3498,True,True,False,False,False,True,True,False,True,False,True,False,False,True,False,False,False,False,True
3499,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3500,True,False,True,True,False,True,False,False,True,True,True,False,True,False,False,False,True,True,True


In [ ]:
nb_sub = nb_sub[(nb_sub['CLUB']==True) | (nb_sub['GYM']==True) |(nb_sub['POOL']==True) | (nb_sub['PARK']==True) | (nb_sub['RWH']==True)]

In [ ]:
nb_final = pd.merge(nb,nb_sub,left_index=True,right_index=True,how='right')
nb_final

,id,amenities,city,gym,latitude,lift,locality,longitude,property_age,society,...,SERVANT,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP
0,ff8081815538c9f70155392e2dd50bdd,"{'LIFT': True, 'GYM': True, 'INTERNET': False,...",CHENNAI,TRUE,13.0478183,TRUE,Saligramam,80.19871,0,CERUS' APARTMENTS,...,False,True,False,True,False,True,True,True,True,True
2,ff8081816354aa09016357a0b4f674c2,"{'LIFT': True, 'GYM': True, 'INTERNET': True, ...",CHENNAI,TRUE,13.0190537,TRUE,Raja Annamalai Puram,80.2567201,16,10 Downing,...,False,True,False,False,True,True,False,True,True,False
4,ff80818160792e8a0160871933f3629b,"{'INTERCOM': True, 'AC': False, 'RWH': True, '...",CHENNAI,FALSE,12.992641,TRUE,Adyar,80.2544098,3,TNHB Legacy Towers,...,False,True,False,False,False,True,False,False,True,True
6,8a9f9b827ece7da4017ececf3732302b,"{'LIFT': True, 'GYM': False, 'INTERNET': False...",CHENNAI,FALSE,12.8982391,TRUE,Sholinganallur,80.2354211,1,1500 MSB,...,False,True,False,False,True,True,False,True,True,True
7,ff8081816a151c04016a15c19e17100d,"{'LIFT': True, 'GYM': False, 'INTERNET': True,...",CHENNAI,FALSE,13.1084942,TRUE,Korattur,80.1876812,0,222 Castle,...,False,True,False,False,True,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,8a9fb7827d3c2857017d3c7941433048,"{'LIFT': True, 'GYM': True, 'INTERNET': False,...",CHENNAI,TRUE,13.0749597,TRUE,Koyambedu,80.1898125,3,Yuga Kalpataru Apartment,...,False,True,False,False,False,False,False,False,False,False
3496,ff8081816243bef4016243fd41061c71,"{'LIFT': True, 'GYM': False, 'INTERNET': True,...",CHENNAI,FALSE,12.7550034,TRUE,Kelambakkam,80.2075017,3,YugaHomes,...,False,True,True,False,True,False,False,False,False,False
3497,ff8081816d3db7e8016d3e1603223d43,"{'INTERCOM': False, 'AC': True, 'RWH': True, '...",CHENNAI,TRUE,12.96218,FALSE,Pallavaram,80.151545,3,Zamin Pallavaram,...,False,True,False,False,False,True,True,False,False,False
3498,ff80818166709ee4016671da157c3f18,"{'LIFT': True, 'GYM': True, 'INTERNET': False,...",CHENNAI,TRUE,13.0550807,TRUE,virugambakkam,80.1904357,10,zebras garden,...,False,True,False,False,True,False,False,False,False,True


In [ ]:
nb_final = nb_final[['society','locality','latitude','longitude']]

In [ ]:
key = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
range = "Uniview Accounts"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
uniview = get_data_df(key, range)

In [ ]:
uniview = uniview[['city','society_name','location_details ‚Üí latitude','location_details ‚Üí longitude']]

In [ ]:
uniview.rename(columns = {'society_name':'society','location_details ‚Üí latitude':'latitude','location_details ‚Üí longitude':'longitude','city':'locality'},inplace=True)
uniview.shape

(5022, 4)

In [ ]:
merge = pd.concat([builder_soc, nb_final, uniview])

In [ ]:
merge['Cleaned Society Name'] = merge['society'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')

<ipython-input-54-63cb1d2d9407>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  merge['Cleaned Society Name'] = merge['society'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')


In [ ]:
merge.drop_duplicates(subset = 'Cleaned Society Name',inplace=True)

In [ ]:
dump_name = 'MIS-Tab1-Prospect-A new 16 dec based on last updated date'
DumpDate = "17 April 2023"
lead = pd.read_csv(file_path + DumpDate + "/Veena/"+ dump_name +".csv")
lead = lead[['lead_id','society_name','number_of_apartments','lead_status','created_by_hierarchy','lead_city','vms_software_in_use','erp_software_in_use','last_updated_on','mc_election_date']]
lead['Product in use'] = lead['vms_software_in_use']+","+lead['erp_software_in_use']
lead.drop(['vms_software_in_use','erp_software_in_use'],inplace=True,axis='columns')

<ipython-input-17-595d85bedb2b>:3: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  lead = pd.read_csv(file_path + DumpDate + "/Veena/"+ dump_name +".csv")


In [ ]:
lead['Cleaned Society Name'] = lead['society_name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')
lead = lead[(~lead['Cleaned Society Name'].isna()) & (lead['lead_city']=='chennai') & (lead['number_of_apartments']!=0) &  (lead['number_of_apartments'].notna())]




<ipython-input-18-14a378e6250d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  lead['Cleaned Society Name'] = lead['society_name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')


In [ ]:
lead[lead.groupby('society_name')['number_of_apartments'].transform(max)==lead['number_of_apartments']]
lead.drop_duplicates(subset='society_name',inplace=True)

<ipython-input-19-08548df3c389>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lead.drop_duplicates(subset='society_name',inplace=True)


#**Fuzzy Grouping**

In [ ]:
matches = merge['Cleaned Society Name'].apply(lambda x: process.extractBests(x, merge['Cleaned Society Name'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
merge['match group'] = merge['Cleaned Society Name']

In [ ]:
for i in matches:
  for j in i:
    print(i[0][0])
    print(j[0])
    merge.loc[merge['match group']==j[0],'match group']=i[0][0]

In [ ]:
lead

,lead_id,society_name,number_of_apartments,lead_status,created_by_hierarchy,lead_city,Product in use,Cleaned Society Name
303,2c415c2dbcfd445087df7e946fdb2de2,Vijayashanthi towers,220.0,new,sales_hierarchy,chennai,NaN,VIJAYASHANTHI TOWERS
347,d3a5daa8991549d8afbc3b93d4050d60,Ozone Green,1356.0,new,sales_hierarchy,chennai,NaN,OZONE GREEN
1093,1c14a1f8527148469248574dd31cbbd7,Alaka Palazzo,280.0,meeting_to_be_fixed,agents_hierarchy,chennai,NaN,ALAKA PALAZZO
2891,75b9c687af9d4f82bbadb6172d7ee20b,marutham heritage,300.0,mc_lead_to_be_collected,agents_hierarchy,chennai,NaN,MARUTHAM HERITAGE
4034,3524a771fbfa44d3a59da0e60198d3ff,Creations Divine,70.0,meeting_to_be_fixed,agents_hierarchy,chennai,NaN,CREATIONS DIVINE
...,...,...,...,...,...,...,...,...
450817,8a968684744970a0017449ebd06717c9,SBIOA Unity Enclave M block,1800.0,inprogress,sales_hierarchy,chennai,NaN,SBIOA UNITY ENCLAVE M BLOCK
450850,8a968684744970a0017449ebcd9a1787,Jains Ashraya Apartments Phase 2,150.0,closed_invalid,sales_hierarchy,chennai,NaN,JAINS ASHRAYA APARTMENTS PHASE 2
450856,8a968684744970a0017449ebca991763,Asset Tree Homes,100.0,inprogress,sales_hierarchy,chennai,NaN,ASSET TREE HOMES
450877,8a968684744970a0017449ebca931750,Lake Side,300.0,closed_invalid,sales_hierarchy,chennai,NaN,LAKE SIDE


In [ ]:
merge = merge.groupby('match group').first().reset_index()

In [ ]:
matches_2 = merge['Cleaned Society Name'].apply(lambda x: process.extractOne(x, lead['Cleaned Society Name'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
df=pd.DataFrame()
df['matched_string_from_lead_dump'],df['similarity_score_lead_dump'],df['index12'] = pd.DataFrame(matches_2)['Cleaned Society Name'].str

<ipython-input-67-e7b18571772f>:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['matched_string_from_lead_dump'],df['similarity_score_lead_dump'],df['index12'] = pd.DataFrame(matches_2)['Cleaned Society Name'].str


In [ ]:
final = pd.merge(merge,df['matched_string_from_lead_dump'],left_index=True,right_index=True,how='left')

In [ ]:
final.drop_duplicates(subset = 'match group')

,match group,society,builder,locality,latitude,longitude,possession date,Cleaned Society Name,matched_string_from_lead_dump
0,10 DOWNING CHAITANIYA,10 Downing chaitaniya,None,chennai,13.0834867,80.1594676,None,10 DOWNING CHAITANIYA,10 DOWNING CHAITANIYA
1,111TNHB FLATS VILLIVAKAM,"111Tnhb Flats, Villivakam",None,Villivakkam,13.10373,80.2024026,None,111TNHB FLATS VILLIVAKAM,None
2,132 10 DOWNING,10 Downing,None,Raja Annamalai Puram,13.0190537,80.2567201,None,10 DOWNING,None
3,1500 MSB,1500 MSB,None,Sholinganallur,12.8982391,80.2354211,None,1500 MSB,None
4,222 CASTLE,222 Castle,None,Korattur,13.1084942,80.1876812,None,222 CASTLE,None
...,...,...,...,...,...,...,...,...,...
6721,ZOLO PLAYA,Zolo playa,None,chennai,12.9661286,80.2619462,None,ZOLO PLAYA,ZOLO PLAYA
6722,ZOLO SARVAM,zolo sarvam,None,pallikaranai,12.948498,80.205107,None,ZOLO SARVAM,None
6723,ZOLO VISTA HEIGHTS,SRR Vista Heights,SRR Homes,Thiruvanmiyur,12.98302456,80.25974264,,SRR VISTA HEIGHTS,None
6724,ZSIVIRA CHEMIE MERK PVT LTD,ZSIVIRA CHEMIE MERK PVT LTD,None,chennai,12.9844524,80.2155277,None,ZSIVIRA CHEMIE MERK PVT LTD,ZSIVIRA CHEMIE MERK PVT LTD


In [ ]:
final_df = pd.merge(final,lead,left_on= 'matched_string_from_lead_dump',right_on='Cleaned Society Name',how='left')
final_df.rename({'number_of_apartments':'Flat Count'}, inplace= True, axis = 1)
final_df = final_df.drop_duplicates(subset = ['society','matched_string_from_lead_dump'])

In [ ]:
final_df = final_df[['society','locality', 'latitude', 'longitude','matched_string_from_lead_dump','Flat Count', 'lead_status','created_by_hierarchy', 'lead_city','possession date','Product in use','lead_id','builder']]
final_df = final_df.drop_duplicates()

In [ ]:
final_df.shape

(6726, 13)

In [ ]:
spreadSheetId = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
rangeName = "Repository"

wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()
#wb.values_clear("masterDEMO!A:AC")

# APPEND DATA TO SHEET
df = final_df
set_with_dataframe(ws, df)

#**Extra requirements**

In [ ]:
key = "1B5cpCMbBzjTavulMqXRd5JjZyn6MUPjDxrqjMDeItGw"
range = "g. (e+f)"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
chennai = get_data_df1(key, range)

key = "1iKgiRR-lOQ-zM6v6bh_1P3vkIujVCeBzVVr_nHuoDMU"
range = "Society Master"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
inventory = get_data_df(key, range)
inventory = inventory[inventory['City']=='NOIDA']
inventory=inventory[['Kibana ID', 'Original Closure Number', 'Society Name','societyStatus', 'Live Product', 'Latest Signing Date','Sold Flats','Closure Number (VMS)','Closure Number (ERP)']]

In [ ]:
chennai['Cleaned source_Project'] = chennai['source_Project'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')
inventory['Cleaned society name'] = inventory['Society Name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')

<ipython-input-11-83b23133cfd2>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  chennai['Cleaned source_Project'] = chennai['source_Project'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')
<ipython-input-11-83b23133cfd2>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  inventory['Cleaned society name'] = inventory['Society Name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')


In [ ]:
matches = chennai['Cleaned source_Project'].apply(lambda x: process.extractOne(x, inventory['Cleaned society name'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
chennai = chennai.join(pd.DataFrame(matches)['Cleaned source_Project'], lsuffix='',rsuffix='_match',how='left')

In [ ]:
chennai['inventory match'] = chennai['Cleaned source_Project_match'].apply(lambda x: x[0] if x!=None else x)

In [ ]:
chennai_merged = pd.merge(chennai,inventory,left_on='inventory match',right_on = 'Cleaned society name',how='left')

In [ ]:
lead.columns

Index(['lead_id', 'society_name', 'number_of_apartments', 'lead_status',
       'created_by_hierarchy', 'lead_city', 'last_updated_on',
       'mc_election_date', 'Product in use', 'Cleaned Society Name'],
      dtype='object')

In [ ]:
matches2 = chennai_merged['Cleaned source_Project'].apply(lambda x: process.extractOne(x, lead['Cleaned Society Name'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
chennai_lead_merged = chennai_merged.join(pd.DataFrame(matches2)['Cleaned source_Project'], lsuffix='',rsuffix='_lead_match',how='left')

In [ ]:
chennai_lead_merged['lead match'] = chennai_lead_merged['Cleaned source_Project_lead_match'].apply(lambda x: x[0] if x!=None else x)

In [ ]:
chennai_lead_merged = pd.merge(chennai_lead_merged,lead,left_on='lead match',right_on = 'Cleaned Society Name',how='left')


In [ ]:
chennai_lead_merged.drop_duplicates(subset = 'UID',keep='first',inplace=True)

In [ ]:
key = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
range = "Uniview Accounts"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
election_date = get_data_df(key, range)
election_date['Cleaned society_name'] = election_date['society_name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')

<ipython-input-31-47e379c9359c>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  election_date['Cleaned society_name'] = election_date['society_name'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')


In [ ]:
matches2 = chennai_merged['Cleaned source_Project'].apply(lambda x: process.extractOne(x, election_date['Cleaned society_name'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
chennai_lead_merged1 = chennai_lead_merged.join(pd.DataFrame(matches2)['Cleaned source_Project'], lsuffix='',rsuffix='_source_match',how='left')

In [ ]:
chennai_lead_merged1['Cleaned source_Project_source_match'] = chennai_lead_merged1['Cleaned source_Project_source_match'].replace(np.nan,None)

In [ ]:
chennai_lead_merged1['source match'] = chennai_lead_merged1['Cleaned source_Project_source_match'].apply(lambda x: x if (x==None)  else x[0])

In [ ]:
chennai_lead_merged2 = pd.merge(chennai_lead_merged1,election_date[['Cleaned society_name','election_date']],left_on='source match',right_on = 'Cleaned society_name',how='left')


In [ ]:
chennai_lead_merged2.drop_duplicates(subset = 'UID',keep='first',inplace=True)

In [ ]:
chennai_lead_merged2[['election_date','mc_election_date']] = chennai_lead_merged2[['election_date','mc_election_date']].applymap(lambda x: x if len(str(x))==3 else x.split('T')[0] )

In [ ]:
chennai_lead_merged2[['election_date','mc_election_date']] = chennai_lead_merged2[['election_date','mc_election_date']].apply(pd.to_datetime)

In [ ]:
chennai_lead_merged2['final_election_date'] = chennai_lead_merged2[['election_date','mc_election_date']].max(axis=1)

In [ ]:
chennai_lead_merged2[['FC','number_of_apartments','Sold Flats']] = chennai_lead_merged2[['FC','number_of_apartments','Sold Flats']].apply(pd.to_numeric)
chennai_lead_merged2['final Flat Count'] = chennai_lead_merged2[['FC','number_of_apartments','Sold Flats']].max(axis=1)

In [ ]:
chennai_lead_merged2[['FC','number_of_apartments','Sold Flats','final Flat Count']]

,FC,number_of_apartments,Sold Flats,final Flat Count
0,2880.0,NaN,NaN,2880.0
1,2613.0,2300.0,NaN,2613.0
2,2482.0,1500.0,NaN,2482.0
3,2300.0,1500.0,NaN,2300.0
4,2300.0,1000.0,NaN,2300.0
...,...,...,...,...
6155,NaN,108.0,NaN,108.0
6156,NaN,300.0,NaN,300.0
6157,NaN,NaN,NaN,NaN
6158,NaN,NaN,NaN,NaN


In [ ]:
# chennai_lead_merged2['to search'] = chennai_lead_merged2['source_Project']+','+chennai_lead_merged2['source_Location']
chennai_lead_merged2['to search'] = chennai_lead_merged2['source_Project']+',chennai'
chennai_lead_merged2['Lat Long'] = chennai_lead_merged2['to search'].apply(lambda x: latlong(x))

In [ ]:
import requests
import urllib.parse

def latlong(x):
  url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(x) +'?format=json'
  print(requests.get(url).json())
  return requests.get(url).json()


In [ ]:
spreadSheetId = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
rangeName = "New"

wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()

df = chennai_lead_merged2
set_with_dataframe(ws, df)

hotel


#**MG sites Fuzzy match**


In [ ]:
key = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
range = "New"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
all_sites = get_data_df(key, range)

range = "MG sites"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
mg_sites = get_data_df(key, range)

In [ ]:
all_sites['Cleaned source_Project'] = all_sites['source_Project'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')
mg_sites['Cleaned Society'] = mg_sites['Society'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')

<ipython-input-30-65c9867389af>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  all_sites['Cleaned source_Project'] = all_sites['source_Project'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')
<ipython-input-30-65c9867389af>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  mg_sites['Cleaned Society'] = mg_sites['Society'].str.upper().str.strip().str.replace('[^A-Z0-9 ]', '')


In [ ]:
matches5 = all_sites['Cleaned source_Project'].apply(lambda x: process.extractOne(x, mg_sites['Cleaned Society'], scorer=fuzz.ratio, score_cutoff=75))

In [ ]:
all_sites = all_sites.join(pd.DataFrame(matches5)['Cleaned source_Project'], lsuffix='',rsuffix='_mg_match',how='left')
all_sites['Cleaned source_Project_mg_match'] = all_sites['Cleaned source_Project_mg_match'].replace(np.nan,None)
all_sites['Cleaned source_Project_mg_match'] = all_sites['Cleaned source_Project_mg_match'].apply(lambda x: x if x==None  else x[0])
all_sites = pd.merge(all_sites,mg_sites[['Cleaned Society','Product','Society']],left_on='Cleaned source_Project_mg_match',right_on = 'Cleaned Society',how='left')


In [ ]:
all_sites

,UID,source_Project,source_Location,source_Builder,source_Type,source_Average Price per sft,source_Possession,source_Price,source,source_url,...,final_election_date,final Flat Count,to search,Latitude,Longitude,Cleaned source_Project,Cleaned source_Project_mg_match,Cleaned Society,Product,Society
0,CHE00001,The World Of Joy,"Siruseri,Chennai south,Chennai",Urbanrise,"Studio, 1, 2, 3 BHK Flat, Studio Flats for sal...",₹4.25 K/sq.ft,Under Construction,₹19 L - 57 L,housing,https://housing.com/in/buy/projects/page/29202...,...,,2880,"The World Of Joy,chennai",12.8273888,80.2023893,THE WORLD OF JOY,None,NaN,NaN,NaN
1,CHE00002,Prestige Bella Vista,"Iyyappanthangal, Chennai",Prestige Estates Projects Ltd.,"3, 4 BHK Flats for sale in Iyyappanthangal, Ch...",Price on request,Ready to Move in,₹1.3 Cr - 2.03 Cr,housing,https://housing.com/in/buy/projects/page/33463...,...,2022-10-31 0:00:00,2613,"Prestige Bella Vista,chennai",13.0397546,80.1316621,PRESTIGE BELLA VISTA,PRESTIGE BELLA VISTA,PRESTIGE BELLA VISTA,MG,Prestige Bella Vista
2,CHE00003,Redundant Embassy Residency,"Cheran Nagar, Perumbakkam,Chennai",Redundant,"1.5, 3 BHK Flats for sale in Perumbakkam, Chennai",₹4.80 K/sq.ft,Ready to Move,₹31.19 L - 74.97 L,housing,https://housing.com/in/buy/projects/page/33715...,...,,2482,"Redundant Embassy Residency,chennai",12.896486,80.2059868,REDUNDANT EMBASSY RESIDENCY,None,NaN,NaN,NaN
3,CHE00004,Urbanrise Manhattan Condos,"Padur, Chennai",Urbanrise,"1.5, 2.5, 3.5 BHK Flats for sale in Padur, Che...",₹8.00 K/sq.ft,"Sep, 2023",₹40 L - 1.44 Cr,housing,https://housing.com/in/buy/projects/page/26565...,...,,2300,"Urbanrise Manhattan Condos,chennai",13.0599114,80.2253605,URBANRISE MANHATTAN CONDOS,None,NaN,NaN,NaN
4,CHE00005,Amarprakash Temple Waves,"Thandalam, Chennai",Amarprakash Developers Pvt. Ltd.,"2, 3 BHK Flats for sale in Thandalam, Chennai",₹3.90 K/sq.ft,Ready to Move,₹24.02 L - 38.61 L,housing,https://housing.com/in/buy/projects/page/38090...,...,,2300,"Amarprakash Temple Waves,chennai",12.9766708,80.1065757,AMARPRAKASH TEMPLE WAVES,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6116,CHE06117,Vishnu MC Villas,GST Road,,,,,,magicbricks,,...,,108,"Vishnu MC Villas,chennai",13.0826802,80.2707184,VISHNU MC VILLAS,None,NaN,NaN,NaN
6117,CHE06118,Vishranthi Mahima,Kundrathur,,,,,,magicbricks,,...,,300,"Vishranthi Mahima,chennai",12.9981685,80.2675587,VISHRANTHI MAHIMA,None,NaN,NaN,NaN
6118,CHE06119,Vishwaksenas Tirumala Apartments,T.Nagar,,,,,,magicbricks,,...,,,"Vishwaksenas Tirumala Apartments,chennai",12.9343879,80.1417137,VISHWAKSENAS TIRUMALA APARTMENTS,None,NaN,NaN,NaN
6119,CHE06120,VSU Dhayal Villa,Thoraipakkam,,,,Possession: Dec-2016,,magicbricks,,...,,,"VSU Dhayal Villa,chennai",13.0826802,80.2707184,VSU DHAYAL VILLA,None,NaN,NaN,NaN


In [ ]:
spreadSheetId = "1PXJjQBagNKXeTAfmXk27W_0r9KmRrJkaq1ndxI4a0VY"
rangeName = "New"

wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()

df = all_sites
set_with_dataframe(ws, df)